In [ ]:
from flask import Flask, jsonify, request
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Inisialisasi dataset
dataset_file = 'D:\\BELAJAR PYTHON\\Capstone\\dataset.csv'
dataset = pd.read_csv(dataset_file)

# Inisialisasi model neural network dengan Keras
def create_neural_network_model(input_dim, output_dim):
    model = keras.Sequential([
        layers.Embedding(input_dim=input_dim, output_dim=64),
        layers.LSTM(64),
        layers.Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Tokenisasi dan padding untuk input model neural network
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['Sentence'])
X = tokenizer.texts_to_sequences(dataset['Sentence'])
X = pad_sequences(X)

# One-hot encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(dataset['Type'])
num_classes = len(label_encoder.classes_)
y = keras.utils.to_categorical(y, num_classes=num_classes)

# Bagi dataset menjadi data pelatihan dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model neural network
input_dim = len(tokenizer.word_index) + 1
output_dim = num_classes
model = create_neural_network_model(input_dim, output_dim)

# Latih model dengan data pelatihan
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

model.save('Capstone_style_recommendation_model.h5')

# How to access data from cloud storage in cloud function

def model_prediction(model, Sentence):
  input_sequence = tokenizer.texts_to_sequences([Sentence])
  input_padded = pad_sequences(input_sequence)

  prediction =  model.predict (input_padded)

  predicted_style = label_encoder.inverse_transform([np.argmax(prediction)])

  return predicted_style[0]

sentence_to_predict = "I don't like to be alone."
predicted_style = model_prediction(model, sentence_to_predict)
print(f"The predicted learning style for the sentence is: {predicted_style}")